In [1]:
import os
import pandas as pd
import sqlite3 as sqlite

In [2]:
path_data = "./data/"
path_news_users = os.path.join(path_data, "newsUser.txt")
path_followers = os.path.join(path_data, "UserUser.txt")
path_news = os.path.join(path_data, "news")
path_news_training = os.path.join(path_news, "training")
path_news_test = os.path.join(path_news, "test")

In [3]:
# Create to db
con = sqlite.connect("project_db.db")
cur = con.cursor()

## Fill user table

In [4]:
user_user_df = pd.read_csv(path_followers, sep="\t", header=None)
user_user_df.rename({0: "follower", 1: "user_followed"}, axis=1, inplace=True)
user_user_df.head()

,follower,user_followed
0,507,1
1,1589,1
2,5307,1
3,11421,1
4,13267,1


In [5]:
# Easy to get userId: get unique values for one of these two columns of the UserUser file
for user_id in user_user_df.user_followed.unique():
    cur.execute("INSERT INTO users VALUES({})".format(user_id))

## Fill followers table
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html  
DataFrame.to_sql params:  
if_exists : {‘fail’, ‘replace’, ‘append’}, default ‘fail’

    How to behave if the table already exists.

        fail: Raise a ValueError.
        replace: Drop the table before inserting new values.
        append: Insert new values to the existing table.

index : bool, default True

    Write DataFrame index as a column. Uses index_label as the column name in the table.

In [35]:
user_user_df.rename({"follower": "userId", "user_followed": "userId_followed"}, 
                    axis=1, 
                    inplace=True)
user_user_df.to_sql("followers", con=con, if_exists='append', index=False)

## Fill News and label tables
### News table

In [6]:
def read_news_file(path_file):
    text = ""
    with open(path_file, "r") as f:
        for i, line in enumerate(f):
            line = line.strip()
            if i == 0:
                title = line
            if line:
                text += "{}\n".format(line)
    return (title, text)

In [7]:
news_dict = dict()
for file in os.listdir(path_news_training):
    news_id = int(os.path.splitext(file)[0])
    if news_id not in news_dict.keys():
        news_dict[news_id] = read_news_file(os.path.join(path_news_training, file))   
    
has_label_df = pd.read_csv(os.path.join(path_data, "labels_training.txt"), header=0)

# checking if the number of news having a label is matching 
# between the training folder and the label file
assert len(news_dict.keys()) == len(has_label_df)

In [24]:
for news_id, news_text in news_dict.items():
    cur.execute("""INSERT INTO news VALUES({}, "{}", "{}")""".format(news_id, 
                                                                     news_text[0].replace('"', '""'), 
                                                                     news_text[1].replace('"', '""')))
# explication pour le replace double quote : https://stackoverflow.com/questions/25387537/inserting-a-table-name-into-a-query-gives-sqlite3-operationalerror-near-sy

10
204
46
150
173
106
27
174
162
171
216
26
99
8
11
61
25
227
230
101
115
140
111
136
233
209
175
224
237
118
73
148
112
194
104
222
160
141
74
14
51
214
52
48
169
77
205
30
12
62
22
196
212
123
89
193
217
65
164
66
107
135
91
53
82
211
195
39
45
29
110
191
229
100
232
79
15
49
154
138
161
72
57
223
64
121
147
103
179
102
59
105
81
131
143
13
90
108
190
92
28
177
151
240
78
42
36
88
221
182
225
98
95
19
132
210
146
71
185
200
197
153
134
54
5
122
1
69
109
203
47
188
130
192
21
6
96
3
238
80
208
75
219
31
231
20
32
55
38
159
23
124
85
189
239
67
184
234
172
56
163
94
157
187
84
166
63
35
41
120
126
142
70
215
167
33
226
149
180
119
9
168
198
207
236
165
139
158
152
50
127
37
170


### Label table

In [33]:
label_df = pd.read_csv(os.path.join(path_data, "labels_training.txt"), header=0)
label_df.rename({"doc": "newsId", "class": "label"}, axis=1, inplace=True)
label_df.to_sql("label", con=con, if_exists='append', index=False)

In [34]:
pd.read_sql_query("SELECT * FROM label LIMIT 10", con=con)

,newsId,label
0,1,0
1,3,0
2,5,0
3,6,0
4,8,0
5,9,0
6,10,0
7,11,0
8,12,0
9,13,0


## Propagation

In [37]:
news_user_df = pd.read_csv(path_news_users, sep="\t", header=None)
news_user_df.rename({0: "newsId", 1: "userId", 2: "propaCount"}, 
                    axis=1, 
                    inplace=True)
news_user_df.head()

,newsId,userId,propaCount
0,240,1,1
1,124,2,1
2,162,3,1
3,233,4,1
4,50,5,1


In [38]:
news_user_df.to_sql("propagation", con=con, if_exists="append", index=False)